### 协程  
#### 微线程，是一种用户态的轻量级线程，协程拥有自己的寄存器上下文和栈。

#### def里面只要看到yield关键字，就表示协程

In [13]:
import time

def work1():
    while True:
        print("----work1---")
        yield
        time.sleep(0.5)
        print('-- 1')

def work2():
    while True:
        print("----work2---")
        yield
        time.sleep(0.5)
        print('-- 2')

def work3():
    while True:
        print("----work3---")
        yield
        time.sleep(0.5)
        print('-- 3')

def main():
    w1 = work1()
    w2 = work2()
    w3 = work3()
    # 碰到yield暂停
    while True:
        next(w1)
        next(w2)
        next(w3)

if __name__ == "__main__":
    main()


----work1---
----work2---
----work3---
-- 1
----work1---
-- 2
----work2---
-- 3
----work3---
-- 1
----work1---
-- 2
----work2---
-- 3
----work3---
-- 1
----work1---
-- 2
----work2---
-- 3
----work3---
-- 1
----work1---
-- 2
----work2---
-- 3
----work3---
-- 1
----work1---
-- 2
----work2---
-- 3
----work3---
-- 1
----work1---
-- 2
----work2---
-- 3
----work3---
-- 1
----work1---
-- 2
----work2---
-- 3
----work3---
-- 1
----work1---
-- 2
----work2---
-- 3
----work3---
-- 1
----work1---
-- 2
----work2---
-- 3
----work3---
-- 1
----work1---
-- 2
----work2---
-- 3
----work3---
-- 1
----work1---


Traceback (most recent call last):
  File "D:\python\project\study\venv\Lib\site-packages\gevent\_ffi\loop.py", line 270, in python_check_callback
    def python_check_callback(self, watcher_ptr): # pylint:disable=unused-argument
    
KeyboardInterrupt
2023-10-18T08:14:19Z


KeyboardInterrupt: 

#### asyncio 协程框架

In [4]:
import asyncio

# 定义协程对象
async def hello():
    print('hello')

# 创建协程对象
coro = hello()
# 获取事件循环对象容器
loop = asyncio.get_event_loop()
# 将协程对象转化为task
# task = loop.create_task(coro)
task = asyncio.ensure_future(coro)
# 将task任务扔进事件循环对象中触发
loop.run_until_complete(task)

hello


#### 协程中绑定回调函数

In [6]:
import asyncio, time

async def stop(x):
    await asyncio.sleep(x)
    return '暂停{}秒'.format(x)

# 直接返回结果
def callback(future):
    print('返回结果是:{}'.format(future.result()))

coro = stop(5)
loop = asyncio.get_event_loop()

task = asyncio.ensure_future(coro)
task.add_done_callback(callback)

loop.run_until_complete(task)
# print('返回结果是{}'.format(task.result()))

RuntimeError: This event loop is already running

返回结果是:暂停5秒


#### 协程中并发

In [7]:
import asyncio

async def do_some_work(x):
    print('wait:', x)
    await asyncio.sleep(x)
    return 'wait time {}'.format(x)

coros = [do_some_work(i) for i in range(1, 5)]
tasks = []
for coro in coros:
    tasks.append(asyncio.ensure_future(coro))
# [tasks.append(asyncio.ensure_future(coro for coro in coros))]

loop = asyncio.get_event_loop()
# loop.run_until_complete(asyncio.wait(tasks))   # wait只接受列表参数
loop.run_until_complete(asyncio.gather(*tasks))   # gather 接收不定量参数
for task in tasks:
    print('任务返回结果：', task.result())


RuntimeError: This event loop is already running

wait: 1
wait: 2
wait: 3
wait: 4


#### 创建1000个文件并发

In [ ]:
import asyncio, time

async def write_file(path, x):
    print('正在生成{}个文件'.format(x))
    with open(path, 'w') as f:
        f.write('this is file {}'.format(x))

if __name__ == '__main__':
    start = time.time()
    loop = asyncio.get_event_loop()

    tasks = []
    for i in range(1, 101):
        tasks.append(asyncio.ensure_future(write_file('d:/demo' + str(i) + '.txt', i)))
    loop.run_until_complete(asyncio.wait(tasks))    # wait只接受列表参数

    print('任务消耗时间为：', time.time()-start)

####  1. 异步函数和`async/await`关键字：

In [1]:
import asyncio

async def hello():   # async申明协程异步方法，不会返回结果，返回协程对象
    print("Hello")
    await asyncio.sleep(3)  # 模拟耗时操作
    print("World")

await hello()

Hello
World


#### 2. 事件循环（Event Loop）:

In [2]:
import asyncio

async def hello(name):
    print("Hello",name)
    await asyncio.sleep(3)  # 模拟耗时操作
    print("World")

async def main():
    await asyncio.gather(hello('jason'), hello('rose'), hello('jack'))  # 同时执行多个协程

await main()

Hello jason
Hello rose
Hello jack
World
World
World


#### 3. 并发与阻塞：

In [5]:
import asyncio

async def fetch_data(url):
    print(f'request now {url}')
    await asyncio.sleep(2)
    print(f'request end {url}')
    
async def main():
    tasks= [
        fetch_data("http://example.com"),
        fetch_data("http://example.org"),
        fetch_data("http://example.net")           
    ]
    await asyncio.gather(*tasks)
    
await main()

request now http://example.com
request now http://example.org
request now http://example.net
request end http://example.com
request end http://example.org
request end http://example.net


#### 利用yield实现协程

In [8]:
import time
        
def func1():
    while True:
        yield

def func2():
    g = func1()
    for i in range(10000):
        i+=1
        next(g)

def main():
    start = time.time()
    func2()
    print(time.time()-start)
        
if __name__ == '__main__':
    main()


0.01630687713623047


#### gevent模块的使用  自动切换到gteenlet
#### gevent本身无法识别time.sleep()等不属于该模块内的IO操作，需要导入gevent下的monkey，利用它监测代码中所有的IO。

In [8]:
# 常规方式
import time

def heng(name):
    print("%s 哼" % name)
    time.sleep(2)
    print("%s 哼" % name)

def ha(name):
    print("%s 哈" % name)
    time.sleep(3)
    print("%s 哈" % name)

start = time.time()
heng('egon')
ha('lsb')
stop = time.time()
print("执行时间：", stop - start)

egon 哼
egon 哼
lsb 哈
lsb 哈
执行时间： 5.013768434524536


####  利用gevent模块

In [9]:
from gevent import monkey;monkey.patch_all()  # 监测代码中所有IO
from gevent import spawn
import time

def heng(name):
    print("%s 哼" % name)
    time.sleep(2)
    print("%s 哼" % name)

def ha(name):
    print("%s 哈" % name)
    time.sleep(3)
    print("%s 哈" % name)
    
start = time.time()
s1 = spawn(heng,'egon')
s2 = spawn(ha,'lsb')
s1.join()  # 此处必须加join等待执行结束
s2.join()
stop = time.time()
print("主执行时间：", stop - start)  # 单线程下实现并发，提升效率

egon 哼
lsb 哈
egon 哼
lsb 哈
主执行时间： 3.0132317543029785


####  单线程利用协程实现并发通信

#### 服务端：

In [ ]:
'''
链接和通信都是io密集型操作，我们只需要在这两者之间来回切换其实就能实现并发的效果
服务端监测链接和通信任务，客户端起多线程同时链接服务端
'''
import socket
from gevent import monkey;monkey.patch_all()
from gevent import spawn

def communicate(conn):
    while True:
        try:   
            data = conn.recv(1024)
            if len(data) == 0:
                break
            print(data.decode('utf-8'))
            conn.send('Hello:'.encode("utf-8") + data.upper())
        except ConnectionResetError:
            break
    conn.close()


def server(ip, port, back_log=5):
    s = socket.socket()
    s.bind((ip, port))
    s.listen(back_log)
    while True:
        conn, addr = s.accept()
        spawn(communicate, conn)

if __name__ == '__main__':
    s1 = spawn(server,'127.0.0.1',8088)
    s1.join()
# 原本服务端需要开启500个线程才能跟500个客户端通信，现在只需要一个线程就可以扛住500客户端
# 进程下面开多个线程，线程下面再开多个协程，最大化提升软件运行效率


#### 客户端：

In [6]:
from threading import Thread,current_thread
import socket
def client_main():
    client = socket.socket()
    client.connect(('127.0.0.1', 8088))
    n = 1
    while True:
        data = "%s %s" % (current_thread().name, n)
        n += 1
        client.send(data.encode("utf-8"))
        info = client.recv(1024)
        print(info.decode('utf-8'))

if __name__ == '__main__':
    for i in range(200):
        t = Thread(target=client_main)
        t.start()


Exception ignored in thread started by: <bound method Thread._bootstrap of <HistorySavingThread(IPythonHistorySavingThread, started 13664)>>
Traceback (most recent call last):
  File "D:\Python\Python311\Lib\threading.py", line 995, in _bootstrap
    self._bootstrap_inner()
  File "D:\Python\Python311\Lib\threading.py", line 1042, in _bootstrap_inner
    self._delete()
  File "D:\Python\Python311\Lib\threading.py", line 1074, in _delete
    del _active[get_ident()]
        ~~~~~~~^^^^^^^^^^^^^
KeyError: 2299091610944
Exception in thread Thread-329 (client_main):
Traceback (most recent call last):
  File "D:\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "D:\Python\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\win\AppData\Local\Temp\ipykernel_12660\209300895.py", line 11, in client_main
  File "D:\python\project\study\venv\Lib\site-packages\gevent\_socketcommon.py", line 660, in recv

The history saving thread hit an unexpected error (AttributeError("'int' object has no attribute 'pending'")).History will not be written to the database.



Exception in thread Thread-406 (client_main):
Traceback (most recent call last):
  File "D:\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "D:\Python\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\win\AppData\Local\Temp\ipykernel_12660\209300895.py", line 11, in client_main
  File "D:\python\project\study\venv\Lib\site-packages\gevent\_socketcommon.py", line 660, in recv
    return self._sock.recv(*args)
           ^^^^^^^^^^^^^^^^^^^^^^
ConnectionAbortedError: [WinError 10053] 你的主机中的软件中止了一个已建立的连接。


Exception in thread Thread-407 (client_main):
Traceback (most recent call last):
  File "D:\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "D:\Python\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\win\AppData\Local\Temp\ipykernel_12660\209300895.py", line 11, in client_main
  File "D:\python\project\study\venv\Lib\site-packages\gevent\_socketcommon.py", line 660, in recv
    return self._sock.recv(*args)
           ^^^^^^^^^^^^^^^^^^^^^^
ConnectionAbortedError: [WinError 10053] 你的主机中的软件中止了一个已建立的连接。
Exception in thread Thread-408 (client_main):
Traceback (most recent call last):
  File "D:\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "D:\Python\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\win\AppData\Local\Temp\ipykernel_12660\209300895.py", line 11, in client_main
  File "D:\py

Exception in thread Thread-422 (client_main):
Traceback (most recent call last):
  File "D:\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "D:\Python\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\win\AppData\Local\Temp\ipykernel_12660\209300895.py", line 11, in client_main
  File "D:\python\project\study\venv\Lib\site-packages\gevent\_socketcommon.py", line 660, in recv
    return self._sock.recv(*args)
           ^^^^^^^^^^^^^^^^^^^^^^
ConnectionAbortedError: [WinError 10053] 你的主机中的软件中止了一个已建立的连接。
Exception in thread Thread-423 (client_main):
Traceback (most recent call last):
  File "D:\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "D:\Python\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\win\AppData\Local\Temp\ipykernel_12660\209300895.py", line 11, in client_main
  File "D:\py

...

    self._target(*self._args, **self._kwargs)
  File "C:\Users\win\AppData\Local\Temp\ipykernel_12660\209300895.py", line 11, in client_main
  File "D:\python\project\study\venv\Lib\site-packages\gevent\_socketcommon.py", line 660, in recv
    return self._sock.recv(*args)
           ^^^^^^^^^^^^^^^^^^^^^^
ConnectionAbortedError: [WinError 10053] 你的主机中的软件中止了一个已建立的连接。
Exception in thread Thread-596 (client_main):
Traceback (most recent call last):
  File "D:\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "D:\Python\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\win\AppData\Local\Temp\ipykernel_12660\209300895.py", line 11, in client_main
  File "D:\python\project\study\venv\Lib\site-packages\gevent\_socketcommon.py", line 660, in recv
    return self._sock.recv(*args)
           ^^^^^^^^^^^^^^^^^^^^^^
ConnectionAbortedError: [WinError 10053] 你的主机中的软件中止了一个已建立的连接。
Exception in thread Th

Exception in thread Thread-514 (client_main):
Traceback (most recent call last):
  File "D:\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "D:\Python\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\win\AppData\Local\Temp\ipykernel_12660\209300895.py", line 11, in client_main
  File "D:\python\project\study\venv\Lib\site-packages\gevent\_socketcommon.py", line 660, in recv
    return self._sock.recv(*args)
           ^^^^^^^^^^^^^^^^^^^^^^
ConnectionAbortedError: [WinError 10053] 你的主机中的软件中止了一个已建立的连接。
Exception in thread Thread-511 (client_main):
Traceback (most recent call last):
  File "D:\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "D:\Python\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\win\AppData\Local\Temp\ipykernel_12660\209300895.py", line 11, in client_main
  File "D:\py

    self.run()
  File "D:\Python\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\win\AppData\Local\Temp\ipykernel_12660\209300895.py", line 11, in client_main
  File "D:\python\project\study\venv\Lib\site-packages\gevent\_socketcommon.py", line 660, in recv
    return self._sock.recv(*args)
           ^^^^^^^^^^^^^^^^^^^^^^
ConnectionAbortedError: [WinError 10053] 你的主机中的软件中止了一个已建立的连接。
Exception in thread Thread-548 (client_main):
Traceback (most recent call last):
  File "D:\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "D:\Python\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\win\AppData\Local\Temp\ipykernel_12660\209300895.py", line 11, in client_main
  File "D:\python\project\study\venv\Lib\site-packages\gevent\_socketcommon.py", line 660, in recv
    return self._sock.recv(*args)
           ^^^^^^^^^^^^^^^^^^^^^^
Conne

Exception in thread Thread-519 (client_main):
Traceback (most recent call last):
  File "D:\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "D:\Python\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\win\AppData\Local\Temp\ipykernel_12660\209300895.py", line 11, in client_main
  File "D:\python\project\study\venv\Lib\site-packages\gevent\_socketcommon.py", line 660, in recv
    return self._sock.recv(*args)
           ^^^^^^^^^^^^^^^^^^^^^^
ConnectionAbortedError: [WinError 10053] 你的主机中的软件中止了一个已建立的连接。
Exception in thread Thread-521 (client_main):
Traceback (most recent call last):
  File "D:\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "D:\Python\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\win\AppData\Local\Temp\ipykernel_12660\209300895.py", line 11, in client_main
  File "D:\py

Exception in thread Thread-553 (client_main):
Traceback (most recent call last):
  File "D:\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "D:\Python\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\win\AppData\Local\Temp\ipykernel_12660\209300895.py", line 11, in client_main
  File "D:\python\project\study\venv\Lib\site-packages\gevent\_socketcommon.py", line 660, in recv
    return self._sock.recv(*args)
           ^^^^^^^^^^^^^^^^^^^^^^
ConnectionAbortedError: [WinError 10053] 你的主机中的软件中止了一个已建立的连接。
Exception in thread Thread-538 (client_main):
Traceback (most recent call last):
  File "D:\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "D:\Python\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\win\AppData\Local\Temp\ipykernel_12660\209300895.py", line 11, in client_main
  File "D:\py

### 线程

#### 基于多线程实现并发通信

In [1]:
import socket
from threading import Thread

def communicate(conn, addr):
    # 通信循环
    while True:
        try:
            data = conn.recv(1024)
            if len(data) == 0:
                break
           # print(data)
            conn.send('hello:'.encode('utf-8') + data.upper())
        except ConnectionResetError:
            print("客户端已断开",addr)
            break

def link(ip, port, back_log=5):
    server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server.bind((ip, port))
    server.listen(back_log)
    # 链接循环
    while True:
        conn, addr = server.accept()
        print('新的连接:',addr)
        # 每来一个客户端就开一个线程
        t = Thread(target=communicate, args=(conn, addr))
        t.start()


if __name__ == '__main__':
    s = Thread(target=link, args=('127.0.0.1',8080))
    s.start()


In [ ]:
import socket

client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client.connect(('127.0.0.1', 8080))

while True:
    msg = input(">>:").strip()
    if len(msg) == 0:
        continue
    print('client send:', msg)
    client.send(msg.encode('utf-8'))
    data = client.recv(1024)
    print('client reback:', data)

新的连接: ('127.0.0.1', 51431)
>>:jason
client send: jason
client reback: b'hello:JASON'


#### 线程池的使用

In [4]:
from concurrent.futures import ThreadPoolExecutor,ProcessPoolExecutor
import time
import os
# 实例化池对象
# 不指定参数的情况，默认是当前计算机cpu个数乘以5，也可以指定线程个数
pool = ThreadPoolExecutor(5)
# pool = ProcessPoolExecutor(5)  # 创建进程池，默认是当前计算机的cpu个数，其他与线程池代码一样

def task(n):
    print(n, os.getpid())
    time.sleep(2)
    return n ** 2

if __name__ == '__main__':
    t_ls = []
    for i in range(20):
        future = pool.submit(task, i)
        # print(future)  # 如果在此处直接打印结果，会使得每次执行都要等待结果，变成同步提交
        t_ls.append(future)
    # 关闭池子并且等待池子中所有的任务运行完毕，再去拿列表中的结果
    pool.shutdown()
    for p in t_ls:
        print('>>>:',p.result())
    print('主线程结束')

0 11896
1 11896
2 11896
3 11896
4 11896
56 11896
7 11896
8 11896
 11896
9 11896
10 11896
11 11896
12 11896
13 11896
14 11896
1516 11896
17 11896
18 11896
 11896
19 11896
>>>: 0
>>>: 1
>>>: 4
>>>: 9
>>>: 16
>>>: 25
>>>: 36
>>>: 49
>>>: 64
>>>: 81
>>>: 100
>>>: 121
>>>: 144
>>>: 169
>>>: 196
>>>: 225
>>>: 256
>>>: 289
>>>: 324
>>>: 361
主


#### 线程中的回调函数  一边执行一边获取结果

In [2]:
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
import time
import os

# 实例化池对象
# 不指定参数的情况，默认是当前计算机cpu个数乘以5，也可以指定线程个数
pool = ThreadPoolExecutor(5)  # 创建了一个池子，池子里面有5个线程，进程池和线程池的唯一区别就是这里
# 不指定参数的情况，进程池默认是当前计算机cpu个数
# pool = ProcessPoolExecutor(5)  # 创建了一个池子，池子里面有5个进程

def task(n):
    print(n, os.getpid())
    time.sleep(2)
    return n ** 2
"""
提交任务的方式
    同步:提交任务之后，原地等待任务的返回结果，再继续执行下一步代码
    异步:提交任务之后，不等待任务的返回结果(通过回调函数拿到返回结果并处理)，直接执行下一步操作
"""
# 回调函数:异步提交之后一旦任务有返回结果，自动交给另外一个去执行
def call_back(n):
    print("拿到了结果：%s" % n.result())


if __name__ == '__main__':
    t_list = []
    for i in range(20):
        future = pool.submit(task, i).add_done_callback(call_back)  # 异步提交任务,通过回掉函数拿到执行结果
        t_list.append(future)  # 如果在此处直接打印结果print(future.result())，会使得每次执行等待结果，变成同步提交
    # print(t_list)  # 前面回调函数拿走结果之后，列表里面全剩了None
    pool.shutdown(wait=True)
    print('主线程结束')


0 18088
1 18088
2 18088
3 18088
4 18088
拿到了结果：0
5 18088
拿到了结果：1
6 18088
拿到了结果：4
7 18088
拿到了结果：9
8 18088
拿到了结果：16
9 18088
拿到了结果：25拿到了结果：36
10 18088

11 18088
拿到了结果：49
12 18088
拿到了结果：64
13 18088
拿到了结果：81
14 18088
拿到了结果：100
15 18088
拿到了结果：144
16 18088
拿到了结果：169
17 18088
拿到了结果：196
18 18088
拿到了结果：121
19 18088
拿到了结果：225拿到了结果：324
拿到了结果：289
拿到了结果：256
拿到了结果：361

主线程结束


#### 模拟爬虫：

In [3]:
from concurrent.futures import ThreadPoolExecutor,ProcessPoolExecutor
from threading import current_thread
import time,random,os
import requests


def get(url):
    print('%s GET %s' %(current_thread().name,url))
    time.sleep(3)
    response=requests.get(url)
    if response.status_code == 200:
        res=response.text
    else:
        res='下载失败'
    return res

def parse(future):
    time.sleep(1)
    res=future.result()
    print('%s 解析结果为%s' %(current_thread().name,len(res)))

if __name__ == '__main__':
    urls=[
        'https://www.baidu.com',
        'https://www.sina.com.cn',
        'https://www.tmall.com',
        'https://www.jd.com',
        'https://www.python.org',
        'https://www.openstack.org',
        'https://www.baidu.com',
        'https://www.baidu.com',
        'https://www.baidu.com',

    ]

    p=ThreadPoolExecutor(4)
    
    for url in urls:
        future=p.submit(get,url)
		# 异步调用:提交完一个任务之后,不在原地等待,而是直接执行下一行代码,会导致任务是并发执行的,,结果futrue对象会在任务运行完毕后自动传给回调函数
        future.add_done_callback(parse)  # parse会在任务运行完毕后自动触发,然后接收一个参数future对象

    p.shutdown(wait=True)

    print('主',current_thread().name)


ThreadPoolExecutor-2_0 GET https://www.baidu.com
ThreadPoolExecutor-2_1 GET https://www.sina.com.cn
ThreadPoolExecutor-2_2 GET https://www.tmall.com
ThreadPoolExecutor-2_3 GET https://www.jd.com
ThreadPoolExecutor-2_1 解析结果为416668
ThreadPoolExecutor-2_1 GET https://www.python.org
ThreadPoolExecutor-2_0 解析结果为2443
ThreadPoolExecutor-2_0 GET https://www.openstack.org
ThreadPoolExecutor-2_2 解析结果为19385
ThreadPoolExecutor-2_2 GET https://www.baidu.com
ThreadPoolExecutor-2_3 解析结果为183630
ThreadPoolExecutor-2_3 GET https://www.baidu.com
ThreadPoolExecutor-2_2 解析结果为2443
ThreadPoolExecutor-2_2 GET https://www.baidu.com
ThreadPoolExecutor-2_3 解析结果为2443
ThreadPoolExecutor-2_1 解析结果为51130
ThreadPoolExecutor-2_0 解析结果为39091
ThreadPoolExecutor-2_2 解析结果为2443
主 MainThread


#### 线程执行顺序

In [5]:
import time, threading
 
# 新线程执行的代码:
def loop():
 
    print('thread %s is running...' % threading.current_thread().name)
 
    n = 0 
    while n < 5: 
        n = n + 1 
        print('thread %s >>> %s' % (threading.current_thread().name, n)) 
        time.sleep(1)
 
    print('thread %s ended.' % threading.current_thread().name)
  
print('thread %s is running...' % threading.current_thread().name)
 
t = threading.Thread(target=loop, name='LoopThread') 
t.start() 
t.join()
 
print('thread %s ended.' % threading.current_thread().name)

thread MainThread is running...
thread LoopThread is running...
thread LoopThread >>> 1
thread LoopThread >>> 2
thread LoopThread >>> 3
thread LoopThread >>> 4
thread LoopThread >>> 5
thread LoopThread ended.
thread MainThread ended.


###  直接连接redis

In [6]:
import redis

r = redis.Redis(host='127.0.0.1', port=6379)
r.set('jason', 45)
print (r.get('jason'))

b'45'


###  连接池redis

In [5]:
import redis

pool = redis.ConnectionPool(host='127.0.0.1', port=6379)
r = redis.Redis(connection_pool=pool)
r.set('foo', 'Bar')
print (r.get('foo'))

b'Bar'


In [1]:
# 通过发布订阅模式的PUB、SUB实现消息队列
# 发布者发布消息到频道了，频道就是一个消息队列。
# 发布者：
import redis
conn = redis.Redis(host='127.0.0.1',port=6379)
conn.publish('104.9MH', "hello")

1

In [ ]:
# 订阅者：
import redis
conn = redis.Redis(host='127.0.0.1',port=6379)
pub = conn.pubsub()
pub.subscribe('104.9MH')
while True:
    msg= pub.parse_response()
    print(msg)

[b'subscribe', b'104.9MH', 1]
